In [4]:
import requests
import pandas as pd
from datetime import datetime
import json
import os
import requests


SECRET_KEY = "a2eb315b0e684dd1c19f6ac6c4bdda7cbaa6a1bb631586f961f96c31a1f403ddcdfccfe5f1c0c260a296319e5be1b58f9df0bf1413077c2e0e7f6be125a323e409cd1b6918bac124c83826a8a617e89be8c9dccc63535a7d13cbf2eba7cefa534d8d5b87d5cdd1602e617fe0d380179da5e8189e907e0c77"
ALL_SHOPS = "31f89356-817d-4a07-abff-6edb45002801,6dd93ef3-e555-4c93-b119-b34b98d68d07,2fb7c502-4694-4f38-ab3c-76ef6a3bc73b,b7889973-6162-4358-a083-04c685404070,ea77b256-1e3d-4e40-9cb9-fd6048669c99"


url = "https://api-admin.billz.ai/v1/auth/login"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json"

}

response = requests.post(url, json={"secret_token": SECRET_KEY}, headers=headers)


data = response.json()
access_token = data["data"]["access_token"]

In [ ]:
import pandas as pd
import sqlite3


print("⏳ Bazadan ma'lumotlar o'qilmoqda...")
conn = sqlite3.connect("transfer.db")

df_history = pd.read_sql("SELECT * FROM d_History", conn)
df_history

In [2]:
import sqlite3
import pandas as pd
import concurrent.futures
from logic import fetch_history_single, DB_FILE

# 1. Sizga kerak bo'lgan barcha Podkategoriyalar ro'yxati (Rasmdan olindi)
TARGET_SUBCATS = [
    'Куртка', 'Полу-водолазка',
    'Свитер'
]

def update_db_for_all_targets():
    conn = sqlite3.connect(DB_FILE)
    
    # 2. Bazadan shu podkategoriyalarga tegishli ID larni yig'ib olamiz
    placeholders = ','.join(['?'] * len(TARGET_SUBCATS))
    query = f"SELECT product_id, Подкатегория FROM d_Mahsulotlar WHERE Подкатегория IN ({placeholders})"
    
    try:
        df_prod = pd.read_sql(query, conn, params=TARGET_SUBCATS)
    except Exception as e:
        print(f"Xatolik: {e}")
        conn.close()
        return

    if df_prod.empty:
        print("❌ Bu podkategoriyalar bo'yicha 'd_Mahsulotlar' jadvalida tovar topilmadi.")
        print("Avval katalogni yangilang (update_catalog_only).")
        conn.close()
        return

    product_ids = df_prod['product_id'].unique().tolist()
    print(f"🚀 Jami {len(product_ids)} ta tovar tarixi tortilmoqda... (Kuting)")

    # 3. Tarixni tortish (Multi-thread)
    all_history = []
    MAX_WORKERS = 10

    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_pid = {executor.submit(fetch_history_single, pid): pid for pid in product_ids}
        count = 0
        for future in concurrent.futures.as_completed(future_to_pid):
            data = future.result()
            if data: all_history.extend(data)
            count += 1
            if count % 100 == 0: 
                print(f"   ⏳ Yuklandi: {count}/{len(product_ids)}")

    # 4. Bazaga yozish (Eskisini o'chirib tashlaymiz, faqat keraklilar qoladi)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM d_History") # Tozalash
    conn.commit()
    
    if all_history:
        df_new = pd.DataFrame(all_history)
        df_new.to_sql("d_History", conn, if_exists="replace", index=False)
        print(f"✅ Baza yangilandi! {len(df_new)} ta qator yozildi.")
    else:
        print("⚠️ Tarix bo'sh keldi.")
        
    conn.close()

# Ishga tushirish
update_db_for_all_targets()

🚀 Jami 3280 ta tovar tarixi tortilmoqda... (Kuting)
   ⏳ Yuklandi: 100/3280
   ⏳ Yuklandi: 200/3280
   ⏳ Yuklandi: 300/3280
   ⏳ Yuklandi: 400/3280
   ⏳ Yuklandi: 500/3280
   ⏳ Yuklandi: 600/3280
   ⏳ Yuklandi: 700/3280
   ⏳ Yuklandi: 800/3280
   ⏳ Yuklandi: 900/3280
   ⏳ Yuklandi: 1000/3280
   ⏳ Yuklandi: 1100/3280
   ⏳ Yuklandi: 1200/3280
   ⏳ Yuklandi: 1300/3280
   ⏳ Yuklandi: 1400/3280
   ⏳ Yuklandi: 1500/3280
   ⏳ Yuklandi: 1600/3280
   ⏳ Yuklandi: 1700/3280
   ⏳ Yuklandi: 1800/3280
   ⏳ Yuklandi: 1900/3280
   ⏳ Yuklandi: 2000/3280
   ⏳ Yuklandi: 2100/3280
   ⏳ Yuklandi: 2200/3280
   ⏳ Yuklandi: 2300/3280
   ⏳ Yuklandi: 2400/3280
   ⏳ Yuklandi: 2500/3280
   ⏳ Yuklandi: 2600/3280
   ⏳ Yuklandi: 2700/3280
   ⏳ Yuklandi: 2800/3280
   ⏳ Yuklandi: 2900/3280
   ⏳ Yuklandi: 3000/3280
   ⏳ Yuklandi: 3100/3280
   ⏳ Yuklandi: 3200/3280
✅ Baza yangilandi! 134492 ta qator yozildi.


In [8]:
import pandas as pd
import sqlite3
from datetime import datetime

# --- SOZLAMALAR ---
DB_FILE = "transfer.db"

SHOP_MAP = {
    "31f89356-817d-4a07-abff-6edb45002801": "Dressco Integro",
    "b7889973-6162-4358-a083-04c685404070": "ANDALUS",
    "2fb7c502-4694-4f38-ab3c-76ef6a3bc73b": "BERUNIY MEN",
    "ea77b256-1e3d-4e40-9cb9-fd6048669c99": "MAGNIT MEN",
    "6dd93ef3-e555-4c93-b119-b34b98d68d07": "SHAXRISTON",
}
MY_SHOP_IDS = list(SHOP_MAP.keys())

def run_monthly_net_sales():
    print("⏳ Analiz boshlandi (Vozvrat + Narx O'zgarishi)...")

    conn = sqlite3.connect(DB_FILE)
    try:
        df_prod = pd.read_sql("SELECT * FROM d_Mahsulotlar", conn)
        df_hist = pd.read_sql("SELECT * FROM d_History", conn)
    except Exception as e:
        print(f"❌ Xatolik: {e}")
        conn.close(); return
    conn.close()

    if df_hist.empty: 
        print("❌ Tarix bo'sh."); return

    # --- 1. DATA TOZALASH (MUHIM: ID LARNI TO'G'RILASH) ---
    # Agar bitta jadvalda ID raqam, ikkinchisida matn bo'lsa, birlashmaydi.
    # Shuning uchun ikkalasini ham matnga o'tkazamiz.
    df_hist['product_id'] = df_hist['product_id'].astype(str).str.strip()
    df_prod['product_id'] = df_prod['product_id'].astype(str).str.strip()

    df_hist['sana'] = pd.to_datetime(df_hist['sana'])
    df_hist['miqdor'] = pd.to_numeric(df_hist['miqdor'], errors='coerce').fillna(0)
    df_hist['type_lower'] = df_hist['turi'].astype(str).str.lower()

    # --- 2. IMPORT SANASINI ANIQLASH ---
    mask_import = (df_hist['to_shop_id'].isin(MY_SHOP_IDS) & (~df_hist['from_shop_id'].isin(MY_SHOP_IDS)))
    grp_import = df_hist[mask_import].groupby('product_id')['sana'].max().reset_index()
    grp_import.columns = ['product_id', 'Latest_Import_Date']

    # --- 3. (YANGI) NARX O'ZGARISH SANASINI ANIQLASH ---
    # Narx o'zgarishiga ishora qiluvchi barcha so'zlar
    search_terms = "price|narx|reprice|repricing|переоценка|update|change"
    mask_reprice = df_hist['type_lower'].str.contains(search_terms, na=False)
    
    grp_reprice = df_hist[mask_reprice].groupby('product_id')['sana'].max().reset_index()
    grp_reprice.columns = ['product_id', 'Narx_Ozgarish_Sanasi']

    # --- 4. SOTUV VA VOZVRATNI AJRATISH ---
    # A) SOTUVLAR (Bizdan ketgan -> Plyus)
    mask_sales = (
        df_hist['type_lower'].str.contains("sale|order|sotuv") &
        df_hist['from_shop_id'].isin(MY_SHOP_IDS)
    )
    df_sales = df_hist[mask_sales].copy()
    df_sales['Real_Shop_ID'] = df_sales['from_shop_id'] # Qaysi do'kondan ketdi
    df_sales['Net_Miqdor'] = df_sales['miqdor']         # Musbat (+)

    # B) VOZVRATLAR (Bizga qaytgan -> Minus)
    mask_returns = (
        df_hist['type_lower'].str.contains("return|vozvrat") &
        df_hist['to_shop_id'].isin(MY_SHOP_IDS)
    )
    df_returns = df_hist[mask_returns].copy()
    df_returns['Real_Shop_ID'] = df_returns['to_shop_id'] # Qaysi do'konga qaytdi
    df_returns['Net_Miqdor'] = -df_returns['miqdor']      # Manfiy (-)

    # C) BIRLASHTIRISH
    df_movements = pd.concat([df_sales, df_returns])

    # Yil va Oy
    df_movements['Yil'] = df_movements['sana'].dt.year
    df_movements['Oy'] = df_movements['sana'].dt.month

    # --- 5. GURUHLASH (NETTO SOTUV) ---
    sales_grouped = df_movements.groupby(['Yil', 'Oy', 'Real_Shop_ID', 'product_id']).agg(
        Netto_Sotuv=('Net_Miqdor', 'sum')
    ).reset_index()

    # Faqat sotuvi 0 dan kattalarni (yoki farqi borlarni) olamiz
    sales_grouped = sales_grouped[sales_grouped['Netto_Sotuv'] != 0]

    # --- 6. DATA BOYITISH (MAGAZIN, PRODUCT INFO, SANALAR) ---
    # Asosiy info
    df_final = pd.merge(sales_grouped, df_prod, on='product_id', how='left')
    # Import sanasi
    df_final = pd.merge(df_final, grp_import, on='product_id', how='left')
    # (YANGI) Narx sanasi
    df_final = pd.merge(df_final, grp_reprice, on='product_id', how='left')

    df_final['Magazin'] = df_final['Real_Shop_ID'].map(SHOP_MAP)

    # --- 7. OY NOMLARI ---
    oy_map = {
        1: "Yanvar", 2: "Fevral", 3: "Mart", 4: "Aprel",
        5: "May", 6: "Iyun", 7: "Iyul", 8: "Avgust",
        9: "Sentabr", 10: "Oktabr", 11: "Noyabr", 12: "Dekabr"
    }
    df_final['Oy_Nomi'] = df_final['Oy'].map(oy_map)

    # --- 8. FORMATLASH ---
    def fmt_date(d): return d.strftime("%d.%m.%Y") if not pd.isnull(d) else "-"
    def fmt_price(p): return f"{int(float(p)):,}".replace(",", " ") if p else "0"

    df_final['Latest_Import_Date'] = df_final['Latest_Import_Date'].apply(fmt_date)
    df_final['Narx_Ozgarish_Sanasi'] = df_final['Narx_Ozgarish_Sanasi'].apply(fmt_date) # <---
    df_final['Sotuv_Narxi'] = df_final['Сена продажи'].apply(fmt_price)
    df_final['Aksiya_Narxi'] = df_final['Акция'].apply(fmt_price)
    # df_final['Hozirgi_Qoldiq'] = df_final['Qoldiq'].fillna(0).astype(int)
    df_final['Artikul_Rang'] = df_final['Артикул'].astype(str) + " " + df_final['Цвет'].astype(str)

    # Ustunlar ketma-ketligi
    cols = [
        'Yil', 'Oy_Nomi', 'Magazin',
        'Netto_Sotuv',  
        'Latest_Import_Date',
        'Narx_Ozgarish_Sanasi', # <--- YANGI USTUN
        'Вид', 'Материал',
        'Aksiya_Narxi', 'Hozirgi_Qoldiq', 'Sotuv_Narxi',
        'Artikul_Rang', 'Поставщик', 'Подкатегория'
    ]
    final_cols = [c for c in cols if c in df_final.columns]

    # Sortirovka
    df_report = df_final[final_cols].sort_values(
        by=['Yil', 'Oy_Nomi', 'Magazin', 'Netto_Sotuv'],
        ascending=[False, False, True, False]
    )

    # Faylga yozish
    fname = f"Netto_Sotuv_Narxli_{datetime.now().strftime('%d%m_%H%M')}.xlsx"
    df_report.to_excel(fname, index=False)
    print(f"✅ Tayyor! Fayl: {fname}")
    print(f"   Jami qatorlar: {len(df_report)}")

if __name__ == "__main__":
    run_monthly_net_sales()

⏳ Analiz boshlandi (Vozvrat + Narx O'zgarishi)...
✅ Tayyor! Fayl: Netto_Sotuv_Narxli_2601_1208.xlsx
   Jami qatorlar: 19880


In [12]:
import pandas as pd
import sqlite3

DB_FILE = "transfer.db"
MY_SHOP_IDS = [
    "31f89356-817d-4a07-abff-6edb45002801",
    "b7889973-6162-4358-a083-04c685404070",
    "2fb7c502-4694-4f38-ab3c-76ef6a3bc73b",
    "ea77b256-1e3d-4e40-9cb9-fd6048669c99",
    "6dd93ef3-e555-4c93-b119-b34b98d68d07"
]

def reprice_sales_by_podcategory():
    conn = sqlite3.connect(DB_FILE)
    df_hist = pd.read_sql("SELECT * FROM d_History", conn)
    df_prod = pd.read_sql("SELECT * FROM d_Mahsulotlar", conn)
    conn.close()

    # ID va sana tozalash
    df_hist['product_id'] = df_hist['product_id'].astype(str).str.strip()
    df_hist['sana'] = pd.to_datetime(df_hist['sana'])
    df_hist['type_lower'] = df_hist['turi'].astype(str).str.lower()
    df_hist['miqdor'] = pd.to_numeric(df_hist['miqdor'], errors='coerce').fillna(0)

    # 1️⃣ Oxirgi narx o'zgarish sanasi
    search_terms = "price|narx|reprice|repricing|переоценка|update|change"
    df_reprice = df_hist[df_hist['type_lower'].str.contains(search_terms, na=False)]
    if df_reprice.empty:
        print("❌ Narx o'zgarish satrlari topilmadi!")
        return pd.DataFrame()

    df_last_reprice = df_reprice.groupby('product_id')['sana'].max().reset_index()
    df_last_reprice.columns = ['product_id', 'Last_Reprice_Date']

    # 2️⃣ Sotuvlar
    df_sales = df_hist[
        df_hist['type_lower'].str.contains("sale|order|sotuv") &
        df_hist['from_shop_id'].isin(MY_SHOP_IDS)
    ].copy()

    # 3️⃣ Oxirgi pereotsinka sanasidan keyin sotuvlar
    df_sales = pd.merge(df_sales, df_last_reprice, on='product_id', how='left')
    df_sales = df_sales[df_sales['sana'] >= df_sales['Last_Reprice_Date']]

    # 4️⃣ Artikul + Rang + Podkategoriya
    df_prod['product_id'] = df_prod['product_id'].astype(str).str.strip()
    df_prod['Artikul_Rang'] = df_prod['Артикул'].astype(str) + " " + df_prod['Цвет'].astype(str)
    df_sales = pd.merge(df_sales, df_prod[['product_id', 'Artikul_Rang', 'Подкатегория']], on='product_id', how='left')

    # 5️⃣ Oy va yil ustunlari
    df_sales['Yil'] = df_sales['sana'].dt.year
    df_sales['Oy'] = df_sales['sana'].dt.month

    # 6️⃣ Oyma-oy sotuvlarni hisoblash
    df_report = df_sales.groupby(['Artikul_Rang', 'Подкатегория', 'Last_Reprice_Date', 'Yil', 'Oy']).agg(
        Sotuv_Soni=('miqdor', 'sum')
    ).reset_index()

    # 7️⃣ Oy nomi qo'shish
    oy_map = {
        1: "Yanvar", 2: "Fevral", 3: "Mart", 4: "Aprel",
        5: "May", 6: "Iyun", 7: "Iyul", 8: "Avgust",
        9: "Sentabr", 10: "Oktabr", 11: "Noyabr", 12: "Dekabr"
    }
    df_report['Oy_Nomi'] = df_report['Oy'].map(oy_map)

    # 8️⃣ Tartib
    df_report = df_report.sort_values(['Artikul_Rang', 'Подкатегория', 'Last_Reprice_Date', 'Yil', 'Oy'])

    return df_report

if __name__ == "__main__":
    df_result = reprice_sales_by_podcategory()
    print(df_result)
    df_result.to_excel("Sotuv_After_Reprice_Podkat.xlsx", index=False)
    print("✅ Fayl tayyor: Sotuv_After_Reprice_Podkat.xlsx")


    Artikul_Rang    Подкатегория   Last_Reprice_Date   Yil  Oy  Sotuv_Soni  \
0         10001           Куртка 2024-11-07 15:44:18  2024  11           5   
1         10001           Куртка 2024-11-07 15:44:21  2024  11          43   
2         10001           Куртка 2024-11-07 15:44:24  2024  11           2   
3         10001           Куртка 2024-11-07 15:44:25  2024  11          45   
4         10001           Куртка 2024-11-07 15:44:28  2024  11          29   
..           ...             ...                 ...   ...  ..         ...   
737   YYO-37637   Полу-водолазка 2025-01-15 18:29:09  2025   1           2   
738   YYO-37637   Полу-водолазка 2025-01-15 18:29:09  2025   2           1   
739   YZB-19471           Куртка 2025-11-02 15:47:17  2025  11           5   
740   YZB-19471           Куртка 2025-11-02 15:47:17  2026   1           1   
741   ZKB-22228           Свитер 2025-08-06 13:30:08  2025   9           3   

     Oy_Nomi  
0     Noyabr  
1     Noyabr  
2     Noyabr  
3  